In [3]:
import random
import time
from pylsl import StreamInfo, StreamOutlet

# Define the stream properties
stream_name = 'MockStream'
stream_type = 'EEG'
channel_count = 24
sampling_rate = 300

# Create a stream outlet
info = StreamInfo(stream_name, stream_type, channel_count, sampling_rate, 'float32', 'myuid12345')
outlet = StreamOutlet(info)

# Generate and publish mock data
while True:
    # Generate a random sample of data
    data = [random.random() for _ in range(channel_count)]

    # Publish the data through the outlet
    outlet.push_sample(data)

    # Wait for a short time to simulate real-time streaming
    time.sleep(1.0 / sampling_rate)

KeyboardInterrupt: 

In [ ]:
import time

from bcipy.acquisition import LslAcquisitionClient
client = LslAcquisitionClient(max_buffer_len=1, save_directory='.')
try:
    seconds = 3
    client.start_acquisition()

    print(
        f"\nCollecting data for {seconds}s... (Interrupt [Ctl-C] to stop)\n"
    )
    time.sleep(seconds)
    samples = client.get_latest_data()
    print(f"Last sample:\n{samples[-1]}")
    print(f"Data written to: {client.recorder.filename}")
    client.stop_acquisition()

except IOError as err:
    print(f'{err.strerror}; make sure you started the server.')
except KeyboardInterrupt:
    print("Keyboard Interrupt")

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/madhav/anaconda3/envs/kedalion_py39/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/madhav/Desktop/bci/kedalion/BciPy/bcipy/acquisition/protocols/lsl/lsl_recorder.py", line 202, in run
    self._init_data_writer(full_metadata)
  File "/home/madhav/Desktop/bci/kedalion/BciPy/bcipy/acquisition/protocols/lsl/lsl_recorder.py", line 125, in _init_data_writer
    check_device(self.device_spec, stream_info)
  File "/home/madhav/Desktop/bci/kedalion/BciPy/bcipy/acquisition/protocols/lsl/lsl_connector.py", line 79, in check_device
    assert device_spec.channel_count == metadata.channel_count(
AssertionError: Channel count error


In [1]:
import struct
import string
import random
import numpy as np

from pylsl import StreamInfo, StreamOutlet
from bcipy.acquisition.devices import DeviceSpec
from bcipy.acquisition.datastream.lsl_server import LslDataServer

device = '/dev/cu.UN-20211209'
blocksize = 0.2
timeout = 5
nchan = 16
fsample = 250

# start_acq = [0x61, 0x7C, 0x87]
# stop_acq = [0x63, 0x5C, 0xC5]
# start_sequence = [0xC0, 0x00]
# stop_sequence = [0x0D, 0x0A]

try:
    # s = serial.Serial(device, 115200, timeout=timeout)
    print("connected to serial port " + device)
except:
    raise RuntimeError("cannot connect to serial port " + device)

lsl_name = 'Unicorn'
lsl_type = 'EEG'
lsl_format = 'float32'
lsl_id = ''.join(random.choice(string.digits) for i in range(6))

device_spec = DeviceSpec(name = lsl_name, )
device_spec.name = lsl_name
device_spec.content_type = lsl_type
device_spec.channel_count = nchan
device_spec.sample_rate = fsample
device_spec.data_type = lsl_format

server = LslDataServer(device_spec=device_spec)

try:
    while True:
        dat = np.zeros(nchan)
        
        #-----------------------READ WITH SERIAL-------------------------#
        # read one block of data from the serial port
        # payload = s.read(45)
        
        #------------------------BATTERY----------------------------------#
        #battery = 100 * float(payload[2] & 0x0F) / 15
        
        
        #------------------------VERIFY PACKET-------------------------#
        # check the start and end bytes
        # if payload[0:2] != b'\xC0\x00':
        #     raise RuntimeError("invalid packet")
        # if payload[43:45] != b'\x0D\x0A':
        #     raise RuntimeError("invalid packet")
        
        #------------------------COLLECT EEG READING-------------------------#
        #eeg = np.zeros(8)
        #for ch in range(0, 8):
            # unpack as a big-endian 32 bit signed integer
            #eegv = struct.unpack('>i', b'\x00' + payload[(3 + ch * 3):(6 + ch * 3)])[0]
            # apply two’s complement to the 32-bit signed integral value if the sign bit is set
            #if (eegv & 0x00800000):
                #eegv = eegv | 0xFF000000
            #eeg[ch] = float(eegv) * 4500000. / 50331642.
            
        #------------------------COLLECT ACCL READING-------------------------#
        #accel = np.zeros(3)
        # unpack as a little-endian 16 bit signed integer
        #accel[0] = float(struct.unpack('<h', payload[27:29])[0]) / 4096.
        #accel[1] = float(struct.unpack('<h', payload[29:31])[0]) / 4096.
        #accel[2] = float(struct.unpack('<h', payload[31:33])[0]) / 4096.
        
        
        #------------------------COLLECT GYRO READING-------------------------#
        #gyro = np.zeros(3)
        # unpack as a little-endian 16 bit signed integer
        #gyro[0] = float(struct.unpack('<h', payload[33:35])[0]) / 32.8
        #gyro[1] = float(struct.unpack('<h', payload[35:37])[0]) / 32.8
        #gyro[2] = float(struct.unpack('<h', payload[37:39])[0]) / 32.8
        
        #------------------------PACKET COUNTER-------------------------------#
        #counter = struct.unpack('<L', payload[39:43])[0]
        
        
        #------------------------RANDOM GENERATOR------------------------------#
        #mimicking unicorn 
        battery = random.uniform(0, 100)
        eeg = np.random.randn(8)
        accel = np.zeros(3)
        gyro = np.zeros(3)
        counter = random.randint(0, 1000)

        # collect the data that will be sent to LSL
        dat[0:8] = eeg
        dat[8:11] = accel
        dat[11:14] = gyro
        dat[14] = battery
        dat[15] = counter

        # send the data to LSL
        server.outlet.push_sample(dat)

        if ((counter % fsample) == 0):
            print('received %d samples, battery %d %%' % (counter, battery))

except:
    print('closing')
    # s.write(stop_acq)
    # s.close()
    server.stop()


connected to serial port /dev/cu.UN-20211209


TypeError: __init__() missing 3 required positional arguments: 'name', 'channels', and 'sample_rate'